# The full urgent care centre call model

**We can now update the model to include the final call centre logic!**

After a patient has spoken to a call operator their priority is triaged.  It is estimated that 40% of patients require a callback from a nurse.  There are 10 nurses available.  A nurse patient consultation has a Uniform distribution lasting between 10 and 20 minutes.

> ⏰ Some call centres run 24/7 while others are open for specified time window during the day.  So we need to cope with **both terminating and non-terminating systems**.

![model image](img/full_model.png "Urgent care call centre")

**Modifications needed**

* Add new default variables for the nurse consultation parameters
* Add new decision variables to `Experiment` for the no. nurses and the consultation distribution.
* Create a second `simpy.Resource` called `nurses` and add it to the simulation model.
* Create the nurse consultation process
* Modify the logic of `service` so that % of patients are called back.
* Collect results and estimate the waiting time for a nurse consultation and nurse utilisation.
* **Bonus:** Add an *optional* warm-up period event.

## 1. Standard Imports

In [7]:
import numpy as np
import pandas as pd
import simpy
import itertools

from abc import ABC, abstractmethod
from typing import Optional, Generator


In [2]:
## 2. Model imports

from model_utility import (
    Distribution,
    Exponential,
    Bernoulli,
    Uniform,
    Triangular,
    Experiment,
    trace,
)

## 3. Modified model code

We will modify the model code and logic that we have already developed to include a nurse consultation for a proportion of the callers.  We create a new function called `nurse_consultation` that contains all the logic. We also need to modify the `service` function so that a proportion of calls are sent to the nurse consultation process.  

In [8]:
class SimplePathwayDelay(ABC):
    """
    Abstract base class representing a simple resource bound delay in a pathway 
    simulation using SimPy.
    
    This class encapsulates the logic for a service activity within a pathway,
    including resource request, service duration, and custom logic before 
    and after service.
    """
    
    def __init__(self, 
                 identifier: int,                 
                 env: simpy.Environment, 
                 resource: simpy.Resource,
                 delay_dist: Distribution,
                 args: Experiment,
                 trace_dp: Optional[int] = 2
    ) -> None:
        """
        Initialize the pathway delay.
        
        Parameters:
        ----------
        activity_name: str
            Name of the activity
            
        identifier: int
            Unique identifier for this delay
            
        env: simpy.Environment
            The simulation environment in which the delay process
            is running.
            
        resource: simpy.resource
            The simpy resource to be requested

        delay_dist: Distribution 
            Used for sampling the process activity duration. Protocol pattern
            must implement the sample method
            
        args: Experiment 
            All model parameters

        trace_dp: int, optional (default=2)
            The number of decimal places that the time is displayed to when
            sending a message to trace.
        """
        self.identifier = identifier
        self.env = env
        self.resource = resource
        self.delay_dist = delay_dist
        self.args = args
        self.trace_dp = trace_dp
        self.waiting_time = -np.inf
        self.service_time = -np.inf
        self.total_time = -np.inf
            
    @abstractmethod
    def before_service(self) -> None:
        """Simulation logic prior to the process entering a service activity"""
        pass

    @abstractmethod
    def after_service(self) -> None:
        """Simulation logic after a service activity has been completed"""
        pass

    def service(self) -> Generator:
        """
        Simulate the service process, including resource request and activity duration.
        
        Returns:
        --------
            A generator that can be yielded in a SimPy environment
        """
        # Record the start time for waiting
        start_wait = self.env.now
        
        # request a resource
        with self.resource.request() as req:
            yield req
    
            # record the waiting time for resource
            self.waiting_time = self.env.now - start_wait
                
            # sample the duration of the activity
            self.service_time = self.delay_dist.sample()
            
            # schedule process to begin again after activity duration
            yield self.env.timeout(self.service_time)
    
    def process(self) -> Generator:
        """
        Run the complete service process, including before and after hooks.
        
        Returns:
            A generator that can be yielded in a SimPy environment
        """
        start_time = self.env.now
        self.before_service()
        yield from self.service()
        self.after_service()
        self.total_time = self.env.now - start_time
        
    def get_process(self) -> simpy.Process:
        """
        Get a SimPy process for this delay that can be started in the 
        environment.
        
        Returns:
        -------
            A SimPy process
        """
        return self.env.process(self.process())


In [ ]:
class NurseConsultationPathway(SimplePathwayDelay):
    """
    Simulates the wait for a consultation with a nurse on the phone.

    Inherits from SimplyPathwayDelay and hence follows this pattern:
    
    1. request and wait for a nurse resource
    2. phone consultation delay
    3. release nurse and exit system
    """
    
    def __init__(self, identifier, env, args):
        super().__init__(
            identifier=identifier,                 
            env=env,
            resource=args.nurses,
            delay_dist=args.nurse_dist,
            args=args)
        
        self.activity_name = "nurse consultation"

    def before_service(self) -> None:
        trace(
            f"self.env.now:.{self.trace_dp}f}: Patient {self.identifier}"
            + f"waiting for nurse call back"
        )

    def after_service(self) -> None:

        # log metrics for process
        self.args.results["nurse_waiting_times"].append(self.waiting_time)
        self.args.results["total_nurse_call_duration"] += self.service_time

        trace(
            f"{self.env.now:.{self.trace_dp}f}: {self.activity_name}"
            + f" for {self.identifier} completed."
        )

In [ ]:
class OperatorProcess(SimplePathwayDelay):
    def __init__(self, identifier, env, args):
        super().__init__(
            identifier=identifier,                 
            env=env,
            resource=args.operators,
            delay_dist=args.operator_dist,
            args=args)
        
        self.activity_name = "Call operator service"

    def before_service(self) -> None:
        trace(
            f"self.env.now:.{self.trace_dp}f}: Patient {self.identifier}"
            + f"waiting for nurse call back"
        )

    def after_service(self) -> None:

        # log metrics for process
        self.args.results["waiting_times"].append(self.waiting_time)
        self.args.results["total_call_duration"] += self.service_time

        trace(
            f"{self.env.now:.{self.trace_dp}f}: {self.activity_name}"
            + f" for {self.identifier} completed."
        )
    
    def service(self):
        """
        simulates the service process for a call operator
    
        1. request and wait for a call operator
        2. phone triage (triangular)
        3. release call operator
        4. a proportion of call continue to nurse consultation
    
        Params:
        ------
        identifier: int
            A unique identifer for this caller
    
        env: simpy.Environment
            The current environent the simulation is running in
            We use this to pause and restart the process after a delay.
    
        args: Experiment
            The settings and input parameters for the current experiment
    
        """
    
        # record the time that call entered the queue
        start_wait = env.now
    
        # request an operator - stored in the Experiment
        with args.operators.request() as req:
            yield req
    
            # record the waiting time for call to be answered
            waiting_time = env.now - start_wait
    
            # store the results for an experiment
            args.results["waiting_times"].append(waiting_time)
            trace(f"operator answered call {identifier} at " + f"{env.now:.3f}")
    
            # the sample distribution is defined by the experiment.
            call_duration = args.call_dist.sample()
    
            # schedule process to begin again after call_duration
            yield env.timeout(call_duration)
    
            # update the total call_duration
            args.results["total_call_duration"] += call_duration
    
            # print out information for patient.
            trace(
                f"call {identifier} ended {env.now:.3f}; "
                + f"waiting time was {waiting_time:.3f}"
            )
    
        # ##########################################################################
        # MODIFICATION NURSE CALL BACK
        # does nurse need to call back?
        # Note the level of the indented code.
        callback_patient = args.callback_dist.sample()
    
        if callback_patient:
            env.process(nurse_consultation(identifier, env, args))
        # ##########################################################################

In [ ]:
def arrivals_generator(env, args):
    """
    IAT is exponentially distributed

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for caller_count in itertools.count(start=1):

        # rhe sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_dist.sample()
        yield env.timeout(inter_arrival_time)

        trace(f"call arrives at: {env.now:.3f}")

        # create a service process
        env.process(service(caller_count, env, args))

## 🥵 Warm-up period

The call centre model starts from empty.  If the call centre runs 24/7 then it is a non-terminating system and our estimates of waiting time and server utilisation are biased due to the empty period at the start of the simulation.  We can remove this initialisation bias using a warm-up period.  

We will implement a warm-up through an **event** that happens once in a single run of the model.  The model will be run for the **warm-up period + results collection period**.  At the end of the warm-up period an event will happen where all variables in the current experiment are reset (e.g. empty lists and set quantitative values to 0.0).

> **Note**: at the point results are reset there are likely resources (call operators and nurses) in use. The result is that we carry over some of the resource usage time from the warm-up to results collection period. It isn't a big deal, but there's potential for resource usage time to be slightly higher than the time scheduled.


In [ ]:
def warmup_complete(warm_up_period, env, args):
    """
    End of warm-up period event. Used to reset results collection variables.

    Parameters:
    ----------
    warm_up_period: float
        Duration of warm-up period in simultion time units

    env: simpy.Environment
        The simpy environment

    args: Experiment
        The simulation experiment that contains the results being collected.
    """
    yield env.timeout(warm_up_period)
    trace(f"{env.now:.2f}: Warm up complete.")
    
    args.init_results_variables()

## 5. Model wrapper functions

Modifications to make to the `single_run` function:

1. Add a warm-up parameters called `wu_period`
1. Create and the nurses resource to the experiment
2. Schedule the `warm_up_complete` process.
3. After the simulation is complete calculate the mean waiting time and mean utilisation for nurses.

In [ ]:
def single_run(
    experiment, 
    rep=0,
    wu_period=WARM_UP_PERIOD, 
    rc_period=RESULTS_COLLECTION_PERIOD
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    wu_period: float, optional (default=WARM_UP_PERIOD)
        The initial transient period of the simulation
        Results from this period are removed from final computations.

    rc_period: float, optional (default=RESULTS_COLLECTION_PERIOD)
        The run length of the model following warm up where results are
        collected.
    """

    # results dictionary.  Each KPI is a new entry.
    run_results = {}

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # we create simpy resource here - this has to be after we
    # create the environment object.
    experiment.operators = simpy.Resource(env, capacity=experiment.n_operators)

    # #########################################################################
    # MODIFICATION: create the nurses resource
    experiment.nurses = simpy.Resource(env, capacity=experiment.n_nurses)
    # #########################################################################

    # we pass the experiment to the arrivals generator
    env.process(arrivals_generator(env, experiment))

    # #########################################################################
    # MODIFICATON: add warm-up period event
    env.process(warmup_complete(wu_period, env, experiment))

    # run for warm-up + results collection period
    env.run(until=wu_period + rc_period)
    # #########################################################################

    # end of run results: calculate mean waiting time
    run_results["01_mean_waiting_time"] = np.mean(
        experiment.results["waiting_times"]
    )

    # end of run results: calculate mean operator utilisation
    run_results["02_operator_util"] = (
        experiment.results["total_call_duration"]
        / (rc_period * experiment.n_operators)
    ) * 100.0

    # #########################################################################
    # MODIFICATION: summary results for nurse process

    # end of run results: nurse waiting time
    run_results["03_mean_nurse_waiting_time"] = np.mean(
        experiment.results["nurse_waiting_times"]
    )

    # end of run results: calculate mean nurse utilisation
    run_results["04_nurse_util"] = (
        experiment.results["total_nurse_call_duration"]
        / (rc_period * experiment.n_nurses)
    ) * 100.0

    # #########################################################################

    # return the results from the run of the model
    return run_results

In [ ]:
def multiple_replications(
    experiment,
    wu_period=WARM_UP_PERIOD,
    rc_period=RESULTS_COLLECTION_PERIOD,
    n_reps=5,
):
    """
    Perform multiple replications of the model.

    Params:
    ------
    experiment: Experiment
        The experiment/paramaters to use with model

    rc_period: float, optional (default=DEFAULT_RESULTS_COLLECTION_PERIOD)
        results collection period.
        the number of minutes to run the model to collect results

    n_reps: int, optional (default=5)
        Number of independent replications to run.

    Returns:
    --------
    pandas.DataFrame
    """

    # loop over single run to generate results dicts in a python list.
    results = [
        single_run(experiment, rep, wu_period, rc_period) 
        for rep in range(n_reps)
    ]

    # format and return results in a dataframe
    df_results = pd.DataFrame(results)
    df_results.index = np.arange(1, len(df_results) + 1)
    df_results.index.name = "rep"
    return df_results

In [ ]:
scenario = Experiment(n_nurses=15, nurse_call_high=30.0)
results = multiple_replications(scenario, wu_period=50.0)
results.describe().round(1).T